# Audio Cleaning
With my handy new legend, I can easily go and select all the audio clips I want. I’ll be applying a voice activity detection filter (VAD) on each clip to help remove some of the background sound.

In [1]:
# Audio Modules
import collections
import contextlib
import sys
import wave
import webrtcvad
# General Modules
import numpy as np
import os
import pandas as pd

# Functions
Audio processing in python is a relativley new thing for me so I "found inspiration" (aka tweeked snippets of code) from the projects of others.
- VAD by [John Wiseman](https://github.com/wiseman)
- Byte to numpy array conversion by [HudsonHuang](https://gist.github.com/HudsonHuang)

In [2]:
def read_wave(path):
    """Reads a .wav file.
    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate

def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)

class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration

def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.
    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.
    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n

def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.
    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.
    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.
    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.
    Arguments:
    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).
    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        #sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                # sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                #sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    #if triggered:
    #    sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    #sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])

def VAD(path,ag=1):
    audio, sample_rate = read_wave(path)
    vad = webrtcvad.Vad(int(ag))
    frames = frame_generator(30, audio, sample_rate)
    frames = list(frames)
    segments = vad_collector(sample_rate, 30, 300, vad, frames)
    
    audio_byte = b"".join(segments)
    return (audio_byte,sample_rate)

# Collection and VAD
Grab the audio clips and apply VAD.

In [3]:
legend = pd.read_csv('Data/CSVs/Audio Legend Raw.csv',index_col='Unnamed: 0')
legend.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8882 entries, 0 to 8881
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   path       8882 non-null   object
 1   original   8882 non-null   object
 2   actorid    8882 non-null   int64 
 3   statment   8882 non-null   int64 
 4   emotion    8882 non-null   object
 5   sex        8882 non-null   object
 6   race       7442 non-null   object
 7   ethnicity  7442 non-null   object
dtypes: int64(2), object(6)
memory usage: 624.5+ KB


In [4]:
# Dataframe to save everything in
audio = pd.DataFrame(columns=['index','original','audio','sample rate','time'])

indices = legend.index
for count,index in enumerate(indices,start=1):
    entry = legend.loc[index]
    track = entry['original']
    path = entry['path']
    try:
        audio_byte,rate = VAD(path+track)
        time = (len(audio_byte)/2)/rate
        # if audio_byte is empty, try less rigorous VAD
        if audio_byte == b'':
            audio_byte,rate = VAD(path+track,ag=0)
            time = (len(audio_byte)/2)/rate
            # if its still empty, input nan
            if audio_byte == b'':
                audio_byte,rate,time = np.nan,np.nan,np.nan
    except:
        audio_byte,rate = np.nan,np.nan
    audio = audio.append({'index':index,'original':track,'audio':audio_byte,'sample rate':rate,
                          'time':time},ignore_index=True)
    # progress bar
    print('%4d of %d complete (%d%%)' % (count,len(indices),(count/len(indices))*100),end='\r')

audio = audio.set_index('index')    
audio.head()

,original,audio,sample rate,time
index,,,,
0,1001_DFA_ANG_XX.wav,b'\xae\x00\xa9\x00\xb1\x00\xac\x00\xa8\x00\xad...,16000,1.68
1,1001_DFA_DIS_XX.wav,b'C\x00\xf3\xff\xdb\xff]\x00\xa8\x00n\x00\x1b\...,16000,1.50
2,1001_DFA_FEA_XX.wav,b'\xd2\x00\xa9\x00\xf1\x00\x8d\x00\xe5\x00\xf6...,16000,1.26
3,1001_DFA_HAP_XX.wav,b'4\xffP\xff\x0e\xff\xb8\xfe\xb3\xfe\'\xff1\xf...,16000,1.29
4,1001_DFA_NEU_XX.wav,b'\xe5\xfe\xf4\xfe\xef\xfe\x06\xff\x19\xff\x14...,16000,1.44


# Cleaning
Merge audio with the legend and drop any clips that seem to be empty

In [5]:
data = legend.merge(audio)
data.head()

,path,original,actorid,statment,emotion,sex,race,ethnicity,audio,sample rate,time
0,Data/Audio/Raw/CREMA_D/,1001_DFA_ANG_XX.wav,1001,3,anger,Male,Caucasian,Not Hispanic,b'\xae\x00\xa9\x00\xb1\x00\xac\x00\xa8\x00\xad...,16000,1.68
1,Data/Audio/Raw/CREMA_D/,1001_DFA_DIS_XX.wav,1001,3,disgust,Male,Caucasian,Not Hispanic,b'C\x00\xf3\xff\xdb\xff]\x00\xa8\x00n\x00\x1b\...,16000,1.50
2,Data/Audio/Raw/CREMA_D/,1001_DFA_FEA_XX.wav,1001,3,fear,Male,Caucasian,Not Hispanic,b'\xd2\x00\xa9\x00\xf1\x00\x8d\x00\xe5\x00\xf6...,16000,1.26
3,Data/Audio/Raw/CREMA_D/,1001_DFA_HAP_XX.wav,1001,3,happy,Male,Caucasian,Not Hispanic,b'4\xffP\xff\x0e\xff\xb8\xfe\xb3\xfe\'\xff1\xf...,16000,1.29
4,Data/Audio/Raw/CREMA_D/,1001_DFA_NEU_XX.wav,1001,3,neutral,Male,Caucasian,Not Hispanic,b'\xe5\xfe\xf4\xfe\xef\xfe\x06\xff\x19\xff\x14...,16000,1.44


Check for the tracks that didn't make it through VAD

In [6]:
tracks_na = data[data['audio'].isna()]
print('%d tracks couldnt be filtered\n' % len(tracks_na))
print('Emotions')
print(tracks_na['emotion'].value_counts())
print('\nActor Gender')
print(tracks_na['sex'].value_counts())

40 tracks couldnt be filtered

Emotions
sad          30
disgust       3
calm          2
fear          2
surprised     1
neutral       1
happy         1
Name: emotion, dtype: int64

Actor Gender
Female    25
Male      15
Name: sex, dtype: int64


Taking a look at some of these tracks, it appears that the voices in the audio clips were too quiet to pick up. This seems to be most common in the `sad` voice clips, with a majority of the actors being women.

In [7]:
# drop the empty clips
data = data.drop(tracks_na.index)

# Save Files
Save all the clean audio clips in the folder `Clean Audio`.

In [8]:
filenames=pd.DataFrame()

indices = data.index
for count,index in enumerate(indices,start=1):
    entry = data.loc[index]
    # Save each file with the legend index as the new filename
    new_name = 'Clip_%04d.wav' % (index)
    path = 'Data/Audio/Clean/'+new_name
    filenames = filenames.append([[entry['original'],new_name]])
    write_wave(path,entry['audio'],entry['sample rate'])
    # progress bar
    print('%4d of %d complete (%d%%)' % (count,len(indices),(count/len(indices))*100),end='\r')

Merge the new filenames with the old filenames and save as the new legend.

In [9]:
# organize filenames
filenames.columns = ['original','filename']
# and merge on the old names
new_legend = data.merge(filenames,how='left',on='original')
new_legend = new_legend.drop(columns=['path','audio','sample rate'])

new_legend.head()

,original,actorid,statment,emotion,sex,race,ethnicity,time,filename
0,1001_DFA_ANG_XX.wav,1001,3,anger,Male,Caucasian,Not Hispanic,1.68,Clip_0000.wav
1,1001_DFA_DIS_XX.wav,1001,3,disgust,Male,Caucasian,Not Hispanic,1.50,Clip_0001.wav
2,1001_DFA_FEA_XX.wav,1001,3,fear,Male,Caucasian,Not Hispanic,1.26,Clip_0002.wav
3,1001_DFA_HAP_XX.wav,1001,3,happy,Male,Caucasian,Not Hispanic,1.29,Clip_0003.wav
4,1001_DFA_NEU_XX.wav,1001,3,neutral,Male,Caucasian,Not Hispanic,1.44,Clip_0004.wav


In [10]:
# perfect now save!
new_legend.to_csv('Data/CSVs/Audio Legend Clean.csv')

### Example Clip
Play a random audio clip from the dataset

In [11]:
import sounddevice as sd
import soundfile as sf

# select a random clip
example = np.random.permutation(new_legend)[0]
example = pd.Series(dict(zip(new_legend.columns,example)))

# print info about the clip
print(example[['filename','statment','emotion']])

# play audio
clip, samplerate = sf.read('Data/Audio/Clean/'+example['filename'])
sd.play(clip, samplerate)
status = sd.wait()

filename    Clip_6015.wav
statment                9
emotion              fear
dtype: object
